In [1]:
import numpy as np
import pandas as pd
import random
import pickle
import json

<h2>Convert rec files for Ranksys</h2>

In [22]:
# filename = f"/home/diego/chat-reranking/experiments/anime/itemid_to_name.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemid_to_name = pickle.load(fp)
# filename = f"/home/diego/chat-reranking/experiments/anime/itemname_to_id.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemname_to_id = pickle.load(fp)

In [23]:
# for i in [16498,31964,30276,20583,32182,33352,24833,34599,13601]:
#     print(f"{i}: {itemid_to_name[i]}")
# itemname_to_id["Assassination Classroom Second Season"]

In [19]:
# print("Tune the length of the candidate set: chatgpt")
# max_pos = []
# for entry in data:
#     base = entry['recs']
#     re_ranked = entry['reranked_recs']
#     if len(re_ranked) == 0:
#         continue
#     
#     def return_pos(i):
#         if i in base:
#             return base.index(i)
#         else:
#             return 0
#         
#     pos = [return_pos(r) for r in re_ranked]
#     max_pos.append(max(pos))
#     
# print(f"Average max reranking pos: {np.mean(max_pos)}") 
# print(f"Average std reranking pos: {np.std(max_pos)}")   

In [113]:
recs_folder = "/home/diego/chat-reranking/experiments/anime/recs/reranked"
recs_name = "chatgpt0613-div-p5-pzt-fold_0"
top_n = 10

# read JSON file
f = open(f"{recs_folder}/{recs_name}.json")
data = json.load(f)
f.close()

In [114]:
train_folder = f"/home/diego/chat-reranking/experiments/anime/fold_0/train_data.csv"
training_data = pd.read_csv(f'{train_folder}', names=["userid", "itemid", "rating"], sep="\t")

Some recommendations might contain 
- less than 10 items
- items that are from the training

We need to remove these items from the recommenations (and count them)

In [115]:
# check whether some recommendations don't intersect with the candidate set
recs = {}
candidate_set = {}
tot_rec_before_pruning = 0
for entry in data:
    userid = entry['userid']
    recs[userid] = entry['reranked_recs']
    candidate_set[userid] = entry['recs']
    tot_rec_before_pruning += len(recs[userid])
    user_train = training_data[training_data["userid"] == entry['userid']]["itemid"].values.tolist()
    to_prune = recs[userid]
    for i, item in enumerate(recs[userid]):
        if item not in candidate_set[userid]:
            del to_prune[i]
            # print(f"user {entry['userid']}: {item}")
            # print(item)
            # print(entry['recs'])
            # print("######")
        recs[userid] = to_prune
print(f"{tot_rec_before_pruning}")

4525


In [116]:
# count users with less than 10 recommendations
count = 0
count_none = 0
avg_len = []
for userid in recs:
    avg_len.append(len(recs[userid]))
    if len(recs[userid]) < 10:
        count += 1
    if len(recs[userid]) == 0:
        count_none += 1
print(f"# of users with less than 10 recommendations: {count}")
print(f"# of users with no recommendations: {count_none}")
print(f"avg number of recommendations per user: {np.mean(avg_len)}")

# of users with less than 10 recommendations: 137
# of users with no recommendations: 22
avg number of recommendations per user: 9.05


In [117]:
# print final recommendations
exclude_violating_recs = True
excluded = 0
with open(f"{recs_folder}/{recs_name}", "w") as f:
    for userid in recs:
        if len(recs[userid]) < 10:
            excluded += 1
            continue
        score = float(top_n)
        usr_str = ""
        for r in recs[userid]:
            usr_str += f"{userid}\t{r}\t{score}\n"
            score -= 1.0
        f.write(usr_str)
print("Printed!")
print(f"# of excluded users: {excluded}")

Printed!
# of excluded users: 137


<h3>Analyse results<h3>

In [55]:
rerankers = ["MMR-pzt-fold_0", "RxQuAD-pzt-fold_0", "xQuAD-pzt-fold_0"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[0]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/baselines/pzt-fold_0_20_100", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/reranked/{rerankers[2]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique()[:300]:
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values[:50])
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}") 
print(f"Average std reranking pos: {np.std(max_pos)}")  

Tune the length of the candidate set: baseline reranker MMR-pzt-fold_0_20_100
Average max reranking pos: 22.436666666666667
Average std reranking pos: 10.324049054943941


In [47]:
re_ranked.head(1)

,userid,itemid,rating
0,198183,27899,10.0


<h2>Results analysis</h2>

In [44]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0-@10.csv")
random_recs = pd.read_csv(f"{results_path}/Random-pzt-fold_0-@10.csv")
gpt_recs = pd.read_csv(f"{results_path}/chatgpt0613-div-p1-pzt-fold_0-@10.csv")
mmr_recs = pd.read_csv(f"{results_path}/MMR-pzt-fold_0-@10.csv")
xquad_recs = pd.read_csv(f"{results_path}/xQuAD-pzt-fold_0-@10.csv")
rxquad_recs = pd.read_csv(f"{results_path}/RxQuAD-pzt-fold_0-@10.csv")

In [45]:
d = {"reranker": ["baseline (MF)", "Random", "chat-gpt", "MMR", "xQuAD", "RxQuAD"]}
print("ChatGPT better than baseline in the following metrics:")
for col in gpt_recs.columns:
    d[col] = [baseline_recs[col][0], random_recs[col][0], gpt_recs[col][0], mmr_recs[col][0], xquad_recs[col][0], rxquad_recs[col][0]]
    if gpt_recs[col][0] > baseline_recs[col][0]:
        print(col)
results = pd.DataFrame(data=d)

ChatGPT better than baseline in the following metrics:
s_recall_norel
eild_norel
binomial_norel
binomial_rel
cbs_norel
epd_norel


In [48]:
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["baseline (MF)", "Random", "chat-gpt", "MMR", "xQuAD", "RxQuAD"]
results

/tmp/ipykernel_119808/726334765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results["reranker"] = ["baseline (MF)", "Random", "chat-gpt", "MMR", "xQuAD", "RxQuAD"]


,prec,ndcg,a_ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,a_ndcg,a_ndcg,reranker
0,0.291400,0.316022,0.324519,0.324519,0.221073,0.775894,0.23365,0.472850,0.324519,0.324519,baseline (MF)
1,0.146800,0.130343,0.153897,0.153897,0.094422,0.810569,0.14475,0.499250,0.153897,0.153897,Random
2,0.251149,0.280032,0.296920,0.296920,0.190156,0.808070,0.21796,0.505244,0.296920,0.296920,chat-gpt
3,0.272000,0.298836,0.313375,0.313375,0.221876,0.874003,0.23715,0.543650,0.313375,0.313375,MMR
4,0.283000,0.301404,0.350005,0.350005,0.211741,0.779803,0.25115,0.538900,0.350005,0.350005,xQuAD
5,0.279600,0.295440,0.337932,0.337932,0.206974,0.769586,0.24370,0.506150,0.337932,0.337932,RxQuAD


In [106]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0-@10.csv")
random_recs = pd.read_csv(f"{results_path}/Random-pzt-fold_0-@10.csv")
chat_p1 = pd.read_csv(f"{results_path}/chatgpt0613-div-p1-pzt-fold_0-@10.csv")
chat_p2 = pd.read_csv(f"{results_path}/chatgpt0613-div-p2-pzt-fold_0-@10.csv")
chat_p3 = pd.read_csv(f"{results_path}/chatgpt0613-div-p3-pzt-fold_0-@10.csv")
chat_p4 = pd.read_csv(f"{results_path}/chatgpt0613-div-p4-pzt-fold_0-@10.csv")
instruct_p1 = pd.read_csv(f"{results_path}/instructgpt-div-p1-pzt-fold_0-@10.csv")
instruct_p2 = pd.read_csv(f"{results_path}/instructgpt-div-p2-pzt-fold_0-@10.csv")

In [107]:
d = {}
for col in chat_p1.columns:
    d[col] = [baseline_recs[col][0], 
              random_recs[col][0], 
              chat_p1[col][0], 
              chat_p2[col][0], 
              chat_p3[col][0], 
              chat_p4[col][0], 
              instruct_p1[col][0],
              instruct_p2[col][0]
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["baseline (MF)", "Random", "chat-gpt-p1", "chat-gpt-p2", "chat-gpt-p3", "chat-gpt-p4", 
                       "instruct-gpt-p1", "instruct-gpt-p2"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.291400,0.316022,0.324519,0.221073,0.775894,0.233650,0.472850,baseline (MF)
1,0.146800,0.130343,0.153897,0.094422,0.810569,0.144750,0.499250,Random
2,0.251149,0.280032,0.296920,0.190156,0.808070,0.217960,0.505244,chat-gpt-p1
3,0.272973,0.297034,0.312735,0.213542,0.811058,0.232601,0.503885,chat-gpt-p2
4,0.265138,0.291274,0.312821,0.204644,0.816447,0.231709,0.513360,chat-gpt-p3
5,0.241768,0.263188,0.285443,0.182893,0.813821,0.215168,0.515244,chat-gpt-p4
6,0.248069,0.266052,0.290308,0.190251,0.810730,0.222693,0.510515,instruct-gpt-p1
7,0.248125,0.274012,0.294177,0.189404,0.812691,0.219271,0.506510,instruct-gpt-p2


In [66]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemid_to_name.pkl"
with open(f"{filename}", 'rb') as fp:
    itemid_to_name = pickle.load(fp)

In [67]:
itemid_to_name[274]

'Sabrina'

In [68]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemname_to_id.pkl"
with open(f"{filename}", 'rb') as fp:
    itemname_to_id = pickle.load(fp)

In [69]:
itemname_to_id["Sabrina"]

486

In [56]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/df_items.csv"
df_items = pd.read_csv(filename)
df_items.head(1)

,movietitle,name,genres
0,Toy Story (1995),Toy Story,"['Animation', 'Childrens', 'Comedy']"


In [58]:
df_items.duplicated().value_counts()

False    1664
True       18
dtype: int64